In [1]:
# IMPORT MODULES #
try:
  import netgen
except ImportError:
  !wget "https://fem-on-colab.github.io/releases/ngsolve-install-real.sh" -O "/tmp/ngsolve-install.sh" && bash "/tmp/ngsolve-install.sh"

try:
  import firedrake
except ImportError:
  !wget "https://fem-on-colab.github.io/releases/firedrake-install-real.sh" -O "/tmp/firedrake-install.sh" && bash "/tmp/firedrake-install.sh"

try:
  import fireshape
except ImportError:
  !pip install git+https://github.com/fireshape/fireshape.git

import matplotlib.pyplot as plt
import numpy as np

import netgen
import ROL
from firedrake import *
from fireshape import *
from netgen.occ import *
from ngsPETSc import NetgenHierarchy
import fireshape.zoo as fsz

/usr/local/lib/python3.10/dist-packages/loopy/schedule/__init__.py:2201: UserWarning: pytools.persistent_dict 'loopy-schedule-cache-v4-2024.1-islpy2024.1-cgen2020.1-967461ba9c3677ee6f0334b9ee95ef0f0e51d208-v1': enabling safe_sync as default. This provides strong protection against data loss, but can be unnecessarily expensive for use cases such as caches.Pass 'safe_sync=False' if occasional data loss is tolerable. Pass 'safe_sync=True' to suppress this warning.
  schedule_cache = WriteOncePersistentDict(
/usr/local/lib/python3.10/dist-packages/loopy/tools.py:914: UserWarning: pytools.persistent_dict 'loopy-memoize-cache-buffer_array-LoopyKeyBuilder.LoopyKeyBuilder-v0-2024.1-islpy2024.1-cgen2020.1-967461ba9c3677ee6f0334b9ee95ef0f0e51d208-v1': enabling safe_sync as default. This provides strong protection against data loss, but can be unnecessarily expensive for use cases such as caches.Pass 'safe_sync=False' if occasional data loss is tolerable. Pass 'safe_sync=True' to suppress this wa

In [2]:
t = 0.12 # specify NACA00xx type

N_x = 100
x = np.linspace(0,1.0089,N_x)

def naca00xx(x,t):
  y = 5*t*(0.2969*(x**0.5) - 0.1260*x - 0.3516*(x**2) + 0.2843*(x**3) - 0.1015*(x**4))
  return np.concatenate((x,np.flip(x)),axis=None), np.concatenate((y,np.flip(-y)),axis=None)

x, y = naca00xx(x,t)

pnts = [Pnt(x[i], y[i], 0) for i in range(len(x))]

spline = SplineApproximation(pnts)
aerofoil = Face(Wire(spline)).Move((0.3,0.5,0)).Rotate(Axis((0.3,0.5,0), Z), -10)
rect = WorkPlane(Axes((-1, 0, 0), n=Z, h=X)).Rectangle(4, 1).Face()
domain = rect - aerofoil

domain.edges.name="wing"
domain.edges.Min(Y).name="bottom"
domain.edges.Max(Y).name="top"
domain.edges.Min(X).name="inlet"
domain.edges.Max(X).name="outlet"
geo = OCCGeometry(domain, dim=2)

ngmesh = geo.GenerateMesh(maxh=1)
ngsolve_mesh = Mesh(ngmesh)

mh = MeshHierarchy(ngsolve_mesh, 2)
mesh = mh[-1]

#fig = plt.figure(figsize=(11,5))
#ax1 = fig.add_subplot(1, 1, 1)
#triplot(mesh,axes=ax1)
#plt.gca().legend()
#plt.show()

In [3]:
class DGMassInv(PCBase):
  def initialize(self, pc):
    _, P = pc.getOperators()
    appctx = self.get_appctx(pc)
    V = dmhooks.get_function_space(pc.getDM())

    # get function spaces
    u = TrialFunction(V)
    v = TestFunction(V)
    massinv = assemble(Tensor(inner(u, v)*dx).inv)
    self.massinv = massinv.petscmat

  def update(self, pc):
    pass

  def apply(self, pc, x, y):
    self.massinv.mult(x, y)
    scaling = 1/float(Re) + float(gamma)
    y.scale(-scaling)

  def applyTranspose(self, pc, x, y):
    raise NotImplementedError("Sorry!")

In [4]:
class solve_navier_stokes(PdeConstraint):
    """Incompressible Navier-Stokes as PDE constraint."""

    def __init__(self, mesh_init, Re, gamma, sp):
        super().__init__()
        self.mesh_init = mesh_init
        self.failed_to_solve = False  # when self.solver.solve() fail
        self.sp = sp
        self.Re = Re
        self.gamma = gamma

        n = FacetNormal(self.mesh_init)
        (x, y) = SpatialCoordinate(self.mesh_init)

        # Define Scott--Vogelius function space W
        self.V = VectorFunctionSpace(self.mesh_init, "CG", 4)
        self.Q = FunctionSpace(self.mesh_init, "DG", 3)
        self.W = MixedFunctionSpace([self.V, self.Q])

        self.bcs = DirichletBC(self.W.sub(0), Constant((0,0)), (1,4,5))

        self.w = Function(self.W, name="Solution")
        (self.u, self.p) = split(self.w)
        (v, q) = split(TestFunction(self.W))

        p0 = 10/13 - x/13 #1atleft,0atright
        #f = Constant((0,-9.81))

        # Define Lagrangian
        L = (
        0.5 * inner(2/self.Re * sym(grad(self.u)), sym(grad(self.u)))*dx
            + inner(dot(self.u,grad(self.u)),self.u)*dx
            -       inner(self.p, div(self.u))*dx
            +       p0 * inner(n, self.u)*ds
            #-       inner(f,self.u)*dx
            + 0.5 * self.gamma * inner(div(self.u), div(self.u))*dx
            )

        # Optimality conditions
        self.F = derivative(L, self.w)

    def solve(self):
        super().solve()
        self.failed_to_solve = False

        w_old = self.w.copy(deepcopy=True)

        try:
            solve(self.F == 0, self.w, self.bcs, solver_parameters=self.sp) # Monitor incompressibility
        except ConvergenceError:
            self.failed_to_solve = True
            self.w.assign(w_old)

        #(u_, p_) = self.w.subfunctions
        #u_.rename("Velocity")
        #p_.rename("Pressure")

        #return u_, p_

In [5]:
sp = {
'mat_type': 'nest',
'snes_monitor': None,
'snes_converged_reason': None,
'snes_max_it': 20,
'snes_atol': 1e-8,
'snes_rtol': 1e-12,
'snes_stol': 1e-06,
'ksp_type': 'fgmres',
'ksp_converged_reason': None, 'ksp_monitor_true_residual': None,
'ksp_max_it': 500,
'ksp_atol': 1e-08,
'ksp_rtol': 1e-10,
'pc_type': 'fieldsplit',
'pc_fieldsplit_type': 'schur', 'pc_fieldsplit_schur_factorization_type': 'full',

'fieldsplit_0': {'ksp_convergence_test': 'skip',
                 'ksp_max_it': 1,
'ksp_norm_type': 'unpreconditioned', 'ksp_richardson_self_scale': False, 'ksp_type': 'richardson',
'pc_type': 'mg',
                 'pc_mg_type': 'full',
                 'mg_coarse_assembled_pc_type': 'lu',
                 'mg_coarse_assembled_pc_factor_mat_solver_type': 'mumps',
                 'mg_coarse_pc_python_type': 'firedrake.AssembledPC',
                 'mg_coarse_pc_type': 'python',
                 'mg_levels': {'ksp_convergence_test': 'skip',
                               'ksp_max_it': 5,
                               'ksp_type': 'fgmres',
                               'pc_python_type': 'firedrake.ASMStarPC',
                               'pc_type': 'python'},
                },
'fieldsplit_1': {'ksp_type': 'preonly',
                 'pc_python_type': __name__ + '.DGMassInv',
                 'pc_type': 'python'},
}

In [6]:
class Objective(ShapeObjective):
    """L2 tracking functional for Poisson problem."""

    def __init__(self, pde_solver: solve_navier_stokes, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.pde_solver = pde_solver

    def value_form(self):
        """Evaluate misfit functional."""

        if self.pde_solver.failed_to_solve:  # return NaNs if state solve fails
            return np.nan * dx(self.pde_solver.mesh_init)
        else:
            w = self.pde_solver.w
            u, p = split(w)
            return inner(grad(u), grad(u)) * dx

In [ ]:
# setup problem
Q = FeControlSpace(mesh)
inner_Q = LaplaceInnerProduct(Q, fixed_bids=[1, 2, 3, 4])
q = ControlVector(Q, inner_Q)

# Define Reynolds number and gamma
Re = Constant(10)
gamma = Constant(10000)

# setup PDE constraint
e = solve_navier_stokes(Q.mesh_m, Re, gamma, sp)

# save state variable evolution in file u2.pvd
#out = fd.File("solution/u2D.pvd")

u = []

def cb():
  return u.append(e.w.split()[0])

# create PDEconstrained objective functional
J_ = Objective(e, Q, cb=cb)
J = ReducedObjective(J_, e)

# add regularization to improve mesh quality
Jq = fsz.MoYoSpectralConstraint(10, Constant(0.5), Q)
J = J + Jq

# ROL parameters
params_dict = {
    'General': {'Print Verbosity': 0,  # set to 1 to understand output
                'Secant': {'Type': 'Limited-Memory BFGS',
                           'Maximum Storage': 10}},
    'Step': {'Type': 'Augmented Lagrangian',
             'Augmented Lagrangian':
             {'Subproblem Step Type': 'Trust Region',
              'Print Intermediate Optimization History': False,
              'Subproblem Iteration Limit': 10}},
    'Status Test': {'Gradient Tolerance': 1e-2,
                    'Step Tolerance': 1e-2,
                    'Constraint Tolerance': 1e-1,
                    'Iteration Limit': 10}}
params = ROL.ParameterList(params_dict, "Parameters")
problem = ROL.OptimizationProblem(J, q)
solver = ROL.OptimizationSolver(problem, params)
solver.solve()

/usr/local/lib/python3.10/dist-packages/firedrake/interpolation.py:385: FutureWarning: The use of `interpolate` to perform the numerical interpolation is deprecated.
This feature will be removed very shortly.

Instead, import `interpolate` from the `firedrake.__future__` module to update
the interpolation's behaviour to return the symbolic `ufl.Interpolate` object associated
with this interpolation.

You can then assemble the resulting object to get the interpolated quantity
of interest. For example,

```
from firedrake.__future__ import interpolate
...

assemble(interpolate(expr, V))
```

Alternatively, you can also perform other symbolic operations on the interpolation operator, such as taking
the derivative, and then assemble the resulting form.

  warnings.warn("""The use of `interpolate` to perform the numerical interpolation is deprecated.


ReducedObjective is deprecated and may be removedin the future. Use PDEconstrainedObjective instead.
  0 SNES Function norm 1.891760872766e-01
    Residual norms for firedrake_1_ solve.
    0 KSP unpreconditioned resid norm 1.891760872766e-01 true resid norm 1.891760872766e-01 ||r(i)||/||b|| 1.000000000000e+00
    1 KSP unpreconditioned resid norm 5.496514554146e-06 true resid norm 5.496514620404e-06 ||r(i)||/||b|| 2.905501799690e-05
    2 KSP unpreconditioned resid norm 4.504991133732e-08 true resid norm 4.505011476025e-08 ||r(i)||/||b|| 2.381385269608e-07
    3 KSP unpreconditioned resid norm 5.248133148688e-10 true resid norm 1.891760872766e-01 ||r(i)||/||b|| 1.000000000000e+00
    Linear firedrake_1_ solve converged due to CONVERGED_ATOL iterations 3
  1 SNES Function norm 3.167943286926e-04
    Residual norms for firedrake_1_ solve.
    0 KSP unpreconditioned resid norm 3.167943286926e-04 true resid norm 3.167943286926e-04 ||r(i)||/||b|| 1.000000000000e+00
    1 KSP unprecondition

/usr/local/lib/python3.10/dist-packages/firedrake/function.py:325: FutureWarning: The .split() method is deprecated, please use the .subfunctions property instead
  warnings.warn("The .split() method is deprecated, please use the .subfunctions property instead", category=FutureWarning)


  Residual norms for firedrake_2_ solve.
  0 KSP unpreconditioned resid norm 7.783940055184e-02 true resid norm 7.783940055184e-02 ||r(i)||/||b|| 1.000000000000e+00
  1 KSP unpreconditioned resid norm 2.120600129101e-04 true resid norm 2.120600135271e-04 ||r(i)||/||b|| 2.724327423178e-03
  2 KSP unpreconditioned resid norm 1.768123046180e-06 true resid norm 1.768115667016e-06 ||r(i)||/||b|| 2.271491885191e-05
  3 KSP unpreconditioned resid norm 2.512663795170e-08 true resid norm 2.514495883872e-08 ||r(i)||/||b|| 3.230363885186e-07
  4 KSP unpreconditioned resid norm 2.383813685047e-10 true resid norm 7.783940055184e-02 ||r(i)||/||b|| 1.000000000000e+00
  Linear firedrake_2_ solve converged due to CONVERGED_ATOL iterations 4
  0 SNES Function norm 5.590079345966e+03
